In [179]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split

In [180]:
matchup = pd.read_csv("matchup.txt", header=None)
matchup = str(matchup[0].values[0])
print(matchup)


BOS vs. LAC


In [181]:
team1 = matchup.split(" ")[0]
team2 = matchup.split(" ")[-1]


# scrape from basketball reference
team1_stats = pd.read_html(f"https://www.basketball-reference.com/teams/{team1}/2023_games.html")
team2_stats = pd.read_html(f"https://www.basketball-reference.com/teams/{team2}/2023_games.html")

team1_stats = team1_stats[0]
team2_stats = team2_stats[0]

In [182]:
team1_stats.dropna(subset={"Tm"}, inplace=True) # null scores haven't happened yet
team1_stats.drop(columns={"Unnamed: 3", "Unnamed: 4", "Unnamed: 8", "Notes"}, inplace=True) # drop columns we don't need
team1_stats = team1_stats[team1_stats["Date"] != "Date"] # remove header row showing up as a game
team1_stats.rename(columns={"Unnamed: 5": "Home", "Unnamed: 7": "Win"}, inplace=True)

# do above steps for team2
team2_stats.dropna(subset={"Tm"}, inplace=True)
team2_stats.drop(columns={"Unnamed: 3", "Unnamed: 4", "Unnamed: 8", "Notes"}, inplace=True)
team2_stats = team2_stats[team2_stats["Date"] != "Date"]
team2_stats.rename(columns={"Unnamed: 5": "Home", "Unnamed: 7": "Win"}, inplace=True)

In [183]:
# function to convert Home column to 0 or 1
def home_to_binary(home):
    if home == "@":
        return 0
    else:
        return 1

# function to convert Win column to 0 or 1
def win_to_binary(win):
    if win == "W":
        return 1
    else:
        return 0

# function to return day of week (first 3 letters) from date
def get_day_of_week(date):
    return date[:3]

team1_stats["Home"] = team1_stats.apply(lambda x: home_to_binary(x["Home"]), axis=1)
team1_stats["Win"] = team1_stats.apply(lambda x: win_to_binary(x["Win"]), axis=1)
team1_stats["Day"] = team1_stats.apply(lambda x: get_day_of_week(x["Date"]), axis=1)

# set data types
team1_stats["Tm"] = team1_stats["Tm"].astype(int)
team1_stats["Opp"] = team1_stats["Opp"].astype(int)
team1_stats["W"] = team1_stats["W"].astype(int)
team1_stats["L"] = team1_stats["L"].astype(int)

# do same for team2
team2_stats["Home"] = team2_stats.apply(lambda x: home_to_binary(x["Home"]), axis=1)
team2_stats["Win"] = team2_stats.apply(lambda x: win_to_binary(x["Win"]), axis=1)
team2_stats["Day"] = team2_stats.apply(lambda x: get_day_of_week(x["Date"]), axis=1)

team2_stats["Tm"] = team2_stats["Tm"].astype(int)
team2_stats["Opp"] = team2_stats["Opp"].astype(int)
team2_stats["W"] = team2_stats["W"].astype(int)
team2_stats["L"] = team2_stats["L"].astype(int)

In [184]:
def day_to_numeric(day):
    if day == "Mon":
        return 1
    if day == "Tue":
        return 2
    if day == "Wed":
        return 3
    if day == "Thu":
        return 4
    if day == "Fri":
        return 5
    if day == "Sat":
        return 6
    if day == "Sun":
        return 7

team1_stats["Day"] = team1_stats.apply(lambda x: day_to_numeric(x["Day"]), axis=1)
team2_stats["Day"] = team2_stats.apply(lambda x: day_to_numeric(x["Day"]), axis=1)

In [185]:
ranks = pd.read_html("https://www.cbssports.com/nba/powerrankings/")
ranks = ranks[0]

ranks["Chg"] = ranks["Chg"].apply(lambda x: 0 if x == "--" else x)
ranks.rename(columns={"Unnamed: 2": "Note"}, inplace=True) # may incorporate sentiment analysis into model

team1_stats["teamname"] = team1_stats["Opponent"].apply(lambda x: x.split(" ")[-1])
team2_stats["teamname"] = team2_stats["Opponent"].apply(lambda x: x.split(" ")[-1])

team1_stats = team1_stats.merge(ranks, how="left", left_on="teamname", right_on="Teams")
team2_stats = team2_stats.merge(ranks, how="left", left_on="teamname", right_on="Teams")

team1_stats = team1_stats[["G", "Home", "W", "L", "Day", "Rk", "Chg", "Tm"]]
team2_stats = team2_stats[["G", "Home", "W", "L", "Day", "Rk", "Chg", "Tm"]]

# impute null values with mean of column
team1_stats.dropna(inplace=True)
team2_stats.dropna(inplace=True)

# export both to csv
team1_stats.to_csv("data/team1_stats.csv", index=False)
team2_stats.to_csv("data/team2_stats.csv", index=False)

In [186]:
X = team1_stats.drop(columns={"Tm"})
y = team1_stats["Tm"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model = LinearRegression()
model.fit(X, y)


LinearRegression()

In [187]:
X = team2_stats.drop(columns={"Tm"})
y = team2_stats["Tm"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model = LinearRegression()
model.fit(X, y)


LinearRegression()

In [188]:
# get matchup to predict
matchup = pd.read_csv("matchup.txt", header=None)
matchup = str(matchup[0].values[0])

# split matchup into team1 and team2
team1 = matchup.split(" ")[0]
team2 = matchup.split(" ")[-1]


In [191]:
team1_stats

,G,Home,W,L,Day,Rk,Chg,Tm
0,1,1,1,0,2,17,5,126
1,2,0,2,0,5,21,3,111
2,3,0,3,0,6,29,0,126
3,4,0,3,1,1,24,2,102
4,5,1,3,2,5,5,1,123
5,6,1,4,2,7,25,4,112
6,7,0,4,3,3,5,1,113
7,8,1,5,3,5,24,2,123
8,9,0,6,3,6,19,4,133
9,10,0,7,3,1,4,3,109
